In [1]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
import time

In [2]:
df = spark.createDataFrame([
    ('id_cliente-1',  'cat-1, cat-2, cat-3'),
    ('id_cliente-2',  'cat-1, cat-4, cat-5'),
    ('id_cliente-3',  'cat-6, cat-7'),
    ('id_cliente-4',  'cat-1, cat-2, cat-7, cat-10'),
    ('id_cliente-5',  'cat-8, cat-10'),
    ('id_cliente-6',  'cat-1, cat-9, cat-10'),
    ('id_cliente-7',  'cat-1, cat-4, cat-5, cat-10'),
    ('id_cliente-8',  'cat-7, cat-9'),
    ('id_cliente-9',  'cat-1'),
    ('id_cliente-10', 'cat-1, cat-2, cat-3, cat-4, cat-5, cat-6, cat-7, cat-8, cat-10')
], ['id_cliente', 'categorias'])

In [3]:
df2 = spark.createDataFrame([
    ('id_cliente-1',  'cat-1, cat-2, cat-3, cat-15'),
    ('id_cliente-2',  'cat-1, cat-4, cat-5, cat-11, cat-14'),
    ('id_cliente-3',  'cat-4, cat-14, cat-15'),
    ('id_cliente-4',  'cat-1, cat-2, cat-7, cat-10'),
    ('id_cliente-5',  'cat-8, cat-10, cat-11'),
    ('id_cliente-6',  'cat-1, cat-9, cat-10, cat-11, cat-13'),
    ('id_cliente-7',  'cat-1, cat-4, cat-5, cat-10'),
    ('id_cliente-8',  'cat-7, cat-9, cat-12, cat-13, cat-14'),
    ('id_cliente-9',  'cat-2'),
    ('id_cliente-10', 'cat-1, cat-2, cat-3, cat-4, cat-5, cat-6, cat-7, cat-8, cat-10')
], ['id_cliente', 'categorias'])

In [4]:
start = time.time()
# Seu script
print(time.time() - start)

6.914138793945312e-05

In [5]:
df.show()

+-------------+--------------------+
 id_cliente| categorias|
+-------------+--------------------+
 id_cliente-1| cat-1, cat-2, cat-3|
 id_cliente-2| cat-1, cat-4, cat-5|
 id_cliente-3| cat-6, cat-7|
 id_cliente-4|cat-1, cat-2, cat...|
 id_cliente-5| cat-8, cat-10|
 id_cliente-6|cat-1, cat-9, cat-10|
 id_cliente-7|cat-1, cat-4, cat...|
 id_cliente-8| cat-7, cat-9|
 id_cliente-9| cat-1|
id_cliente-10|cat-1, cat-2, cat...|
+-------------+--------------------+

In [6]:
df2.show()

+-------------+--------------------+
 id_cliente| categorias|
+-------------+--------------------+
 id_cliente-1|cat-1, cat-2, cat...|
 id_cliente-2|cat-1, cat-4, cat...|
 id_cliente-3|cat-4, cat-14, ca...|
 id_cliente-4|cat-1, cat-2, cat...|
 id_cliente-5|cat-8, cat-10, ca...|
 id_cliente-6|cat-1, cat-9, cat...|
 id_cliente-7|cat-1, cat-4, cat...|
 id_cliente-8|cat-7, cat-9, cat...|
 id_cliente-9| cat-2|
id_cliente-10|cat-1, cat-2, cat...|
+-------------+--------------------+

In [7]:
df.select('categorias').distinct().show()

+--------------------+
 categorias|
+--------------------+
cat-1, cat-9, cat-10|
cat-1, cat-2, cat...|
 cat-8, cat-10|
 cat-1, cat-2, cat-3|
cat-1, cat-4, cat...|
 cat-7, cat-9|
 cat-6, cat-7|
 cat-1, cat-4, cat-5|
 cat-1|
cat-1, cat-2, cat...|
+--------------------+

In [8]:
from pyspark.sql.functions import col, split
import pyspark.sql.functions as f

In [9]:
df4=df.select(
        col("id_cliente"), substring_index(col("id_cliente"), "-", -1).cast(IntegerType()).alias("id"),
        f.split("categorias", ", ").alias("categorias"), 
        f.posexplode(f.split("categorias", ", ")).alias("pos", "categoriasx"),
        substring_index(col("categoriasx"), "-", -1).cast(IntegerType()).alias("categ")
    )\
     .sort(asc("id"),asc("categ")).crosstab('id','categ')
df4.show()

+--------+---+---+---+---+---+---+---+---+---+---+
id_categ| 1| 10| 2| 3| 4| 5| 6| 7| 8| 9|
+--------+---+---+---+---+---+---+---+---+---+---+
 5| 0| 1| 0| 0| 0| 0| 0| 0| 1| 0|
 10| 1| 1| 1| 1| 1| 1| 1| 1| 1| 0|
 1| 1| 0| 1| 1| 0| 0| 0| 0| 0| 0|
 6| 1| 1| 0| 0| 0| 0| 0| 0| 0| 1|
 9| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0|
 2| 1| 0| 0| 0| 1| 1| 0| 0| 0| 0|
 7| 1| 1| 0| 0| 1| 1| 0| 0| 0| 0|
 3| 0| 0| 0| 0| 0| 0| 1| 1| 0| 0|
 8| 0| 0| 0| 0| 0| 0| 0| 1| 0| 1|
 4| 1| 1| 1| 0| 0| 0| 0| 1| 0| 0|
+--------+---+---+---+---+---+---+---+---+---+---+

In [10]:
cols=df4.columns[1:]
cols_cat = []
for i in range(len(cols)):
  aux = int(cols[i])
  cols_cat.append(aux)
cols_cat.sort()

In [11]:
cols_cat_str = []
for i in range(len(cols_cat)):
  aux = str(cols_cat[i])
  cols_cat_str.append(aux)
print(cols_cat_str)

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

In [12]:
cols_final = []
cols_final.append(df4.columns[0])
cols_final.extend(cols_cat_str)
print(cols_final)

['id_categ', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

In [13]:
df5 = df4[cols_final]
df5 = df5.withColumn("id_categ", df5["id_categ"].cast(IntegerType())).sort(asc('id_categ'))
df5.show()

+--------+---+---+---+---+---+---+---+---+---+---+
id_categ| 1| 2| 3| 4| 5| 6| 7| 8| 9| 10|
+--------+---+---+---+---+---+---+---+---+---+---+
 1| 1| 1| 1| 0| 0| 0| 0| 0| 0| 0|
 2| 1| 0| 0| 1| 1| 0| 0| 0| 0| 0|
 3| 0| 0| 0| 0| 0| 1| 1| 0| 0| 0|
 4| 1| 1| 0| 0| 0| 0| 1| 0| 0| 1|
 5| 0| 0| 0| 0| 0| 0| 0| 1| 0| 1|
 6| 1| 0| 0| 0| 0| 0| 0| 0| 1| 1|
 7| 1| 0| 0| 1| 1| 0| 0| 0| 0| 1|
 8| 0| 0| 0| 0| 0| 0| 1| 0| 1| 0|
 9| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0|
 10| 1| 1| 1| 1| 1| 1| 1| 1| 0| 1|
+--------+---+---+---+---+---+---+---+---+---+---+

In [14]:
print(oldColumns)
print(newColumns)


['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
['cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9', 'cat10']

In [15]:
oldColumns = df5.schema.names[1:]

newColumns = []
for name in cols_cat_str:
  newColumns.append('cat'+name)
  
texto = ''
for i in range(len(oldColumns)):
  texto = texto + "'"+oldColumns[i]+"' AS '"+newColumns[i]+"', "
  
texto_final = texto[:-2]
print(texto_final)
  
sqlContext.registerDataFrameAsTable(df5, "myTable")
df6 = sqlContext.sql("SELECT CONCAT('client-uuid-',id_categ) AS id, * from myTable")
df6.show()

'1' AS 'cat1', '2' AS 'cat2', '3' AS 'cat3', '4' AS 'cat4', '5' AS 'cat5', '6' AS 'cat6', '7' AS 'cat7', '8' AS 'cat8', '9' AS 'cat9', '10' AS 'cat10'
+--------------+--------+---+---+---+---+---+---+---+---+---+---+
 id|id_categ| 1| 2| 3| 4| 5| 6| 7| 8| 9| 10|
+--------------+--------+---+---+---+---+---+---+---+---+---+---+
 client-uuid-1| 1| 1| 1| 1| 0| 0| 0| 0| 0| 0| 0|
 client-uuid-2| 2| 1| 0| 0| 1| 1| 0| 0| 0| 0| 0|
 client-uuid-3| 3| 0| 0| 0| 0| 0| 1| 1| 0| 0| 0|
 client-uuid-4| 4| 1| 1| 0| 0| 0| 0| 1| 0| 0| 1|
 client-uuid-5| 5| 0| 0| 0| 0| 0| 0| 0| 1| 0| 1|
 client-uuid-6| 6| 1| 0| 0| 0| 0| 0| 0| 0| 1| 1|
 client-uuid-7| 7| 1| 0| 0| 1| 1| 0| 0| 0| 0| 1|
 client-uuid-8| 8| 0| 0| 0| 0| 0| 0| 1| 0| 1| 0|
 client-uuid-9| 9| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0|
client-uuid-10| 10| 1| 1| 1| 1| 1| 1| 1| 1| 0| 1|
+--------------+--------+---+---+---+---+---+---+---+---+---+---+

In [16]:
df6=df5.select(
        col("id_categ").alias("id"),
        col("*")
    )
df6.show()

+---+--------+---+---+---+---+---+---+---+---+---+---+
 id|id_categ| 1| 2| 3| 4| 5| 6| 7| 8| 9| 10|
+---+--------+---+---+---+---+---+---+---+---+---+---+
 1| 1| 1| 1| 1| 0| 0| 0| 0| 0| 0| 0|
 2| 2| 1| 0| 0| 1| 1| 0| 0| 0| 0| 0|
 3| 3| 0| 0| 0| 0| 0| 1| 1| 0| 0| 0|
 4| 4| 1| 1| 0| 0| 0| 0| 1| 0| 0| 1|
 5| 5| 0| 0| 0| 0| 0| 0| 0| 1| 0| 1|
 6| 6| 1| 0| 0| 0| 0| 0| 0| 0| 1| 1|
 7| 7| 1| 0| 0| 1| 1| 0| 0| 0| 0| 1|
 8| 8| 0| 0| 0| 0| 0| 0| 1| 0| 1| 0|
 9| 9| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0|
 10| 10| 1| 1| 1| 1| 1| 1| 1| 1| 0| 1|
+---+--------+---+---+---+---+---+---+---+---+---+---+